# BERT as Benchmark

- Fine tune BERT model using autotrain from Huggingface: https://huggingface.co/autotrain
- Used the train.csv and val.csv in the autotrain job
- Use default configuration
- Fine-tuned model : https://huggingface.co/vincentclaes/autotrain-0br8k-gdjpm
- accuracy: 0.9994418604651163

## Run against test dataset and calculate the accuracy

In [2]:
!pip install transformers scikit-learn --quiet

In [5]:
from transformers import pipeline

# Load the model (if you're using the model from the Hugging Face Hub)
classifier_zero_shot = pipeline('zero-shot-classification', model='google-bert/bert-base-uncased', truncation=True)
classifier_fine_tuned = pipeline('text-classification', model='vincentclaes/autotrain-yelp-2likv', truncation=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/vincent/Workspace/classification_with_llm/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a de

In [6]:
# read test.csv, and make prediction for the column 'question' and add the result to the column 'predicted_class_name'
import pandas as pd

df = pd.read_csv('test.csv')


# LABELS = ["1 star review", "2 star review", "3 star review", "4 star review", "5 star review"]
LABEL_MAPPING = {1: "1 star review", 2: "2 star review", 3: "3 star review", 4: "4 star review", 5: "5 star review"}

df_zero_shot = df.copy()
df_zero_shot['predicted_class_name'] = df_zero_shot['question'].apply(lambda x: classifier_zero_shot(x, candidate_labels=list(LABEL_MAPPING.keys()))["labels"][0])

df_fine_tuned = df.copy()
df_fine_tuned['predicted_class_name'] = df_fine_tuned['question'].apply(lambda x: classifier_fine_tuned(x)[0]['label'])

In [12]:
def calculate_accuracy(df):
    from datasets import load_metric
    
    metric = load_metric("accuracy")
    accuracy = metric.compute(predictions=df['predicted_class_name'], references=df['class_name'])
    
    print(f"Accuracy: {accuracy['accuracy']}")
    
    return accuracy

zero_shot_acc = calculate_accuracy(df_zero_shot)
print(f"Zero-Shot Accuracy: {zero_shot_acc}")
fine_tuned_acc = calculate_accuracy(df_fine_tuned)
print(f"Fine-Tuned Accuracy: {fine_tuned_acc}")

Accuracy: 0.18666666666666668
Zero-Shot Accuracy: {'accuracy': 0.18666666666666668}
Accuracy: 0.29904761904761906
Fine-Tuned Accuracy: {'accuracy': 0.29904761904761906}
